In [16]:
import torch
import numpy as np
import matplotlib

In [17]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-01-04 11:55:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2024-01-04 11:55:41 (24.5 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [73]:
# HYPER PARMETARS
batch_size = 16 #Kad se kaze batch misli se na koliko sekvenci paralelno cemo gledati sum gradient
block_size = 32 #Koliko karaktera prije cemo gledati
max_iter = 1000 #Koliko training loop
eval_interval = 100 #Kad se vrsi provjera gubitaka
learning_rate = 1e-3
eval_iters = 200
n_embeds = 64
n_layer = 8
n_heads = 4
dropout = 0

#

with open("input.txt", "r", encoding="utf-8") as f:
  text =f.read()

vocab_size  = 255


In [47]:
def encoder(text):
  encoded = []
  for i in range(len(text)):
    encoded.append(ord(text[i]))
  return encoded
#encoded = encoder(text)

def decoder(encoded):
  decoded = []
  for i in range(len(encoded)):
    decoded.append(chr(encoded[i]))
  return decoded
encoded = encoder(text)
data = torch.tensor(encoded)  # Data pretstavlja tensor koji cemo koristiti
n = int(len(data)*0.9) #90% training 10% validacija
trainig_data=data[:n]
val_data=data[n:]
#Zavisi koji je split training ili validacijoni
def get_batch(split):
  data = trainig_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  #x, y = x.to(device), y.to(device)
  return x, y
#x,y = get_batch("train")


In [74]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class Head(torch.nn.Module):
  # ---------------------------------------- #
  def __init__(self, head_size) -> None:   #Prima n_heads u konstruktoru register_baffer je nasledjeno iz Module
     super().__init__()
     self.key = torch.nn.Linear(n_embeds,head_size,bias = False)
     self.query  = torch.nn.Linear(n_embeds,head_size,bias = False)
     self.value = torch.nn.Linear(n_embeds,head_size,bias = False)
     self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
     self.dropout = torch.nn.Dropout(dropout)
  # ---------------------------------------- #
  def forward(self,x):
    B,T,C = x.shape   #Vadi batch, time, chanels
    k = self.key(x)   #DIM B,T,C
    q = self.query(x) #DIM B,T,C
    wei = q @ k.transpose(-2,-1) * C**-0.5 #C**-0.5 Je korijen iz broja dimenzija
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # Da sakrijemo elemente poslije usustini matrica
                                                                  # Gdje su svi brojevi ispod glavne avrage of sum of
    wei = torch.nn.functional.softmax(wei, dim=1)
    wei = self.dropout(wei)     #Valjda pomaze pri treningu Random postavi neki ulaz na nulu
    v = self.value(x) #
    out = wei @ v
    return out
   # ---------------------------------------- #

class MultiHeadAttention(torch.nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):           #Postavlja vise glava za procesovanje
        super().__init__()
        self.heads = torch.nn.ModuleList([Head(head_size) for _ in range(num_heads)]) #Module liste svih modula unutar klasa
        self.proj = torch.nn.Linear(n_embeds, n_embeds)  #Jos jedan layer zbog learninga
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(torch.nn.Module):  #Fead forward model poslije
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(n_embd, 4 * n_embd),
            torch.nn.ReLU(),
            torch.nn.Linear(4 * n_embd, n_embd),
            torch.nn.Dropout(dropout),
        )

    def forward(self, x): #Ide kroz
        return self.net(x)

class Block(torch.nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)   # Multi head atention
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = torch.nn.LayerNorm(n_embd)   # Normalizacija meen 0 var 1
        self.ln2 = torch.nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))    ## + Pomaze rasporediti gradient unutar nn
        x = x + self.ffwd(self.ln2(x))
        return x


class SimpleLanguageModel(torch.nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = torch.nn.Embedding(vocab_size, n_embeds)
        self.position_embedding_table =  torch.nn.Embedding(block_size, n_embeds)
        self.blocks = torch.nn.Sequential(*[Block(n_embeds, n_head=n_heads) for _ in range(n_layer)])
        self.ln_f =  torch.nn.LayerNorm(n_embeds) # final layer norm
        self.lm_head =  torch.nn.Linear(n_embeds, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T)) #
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss =  torch.nn.functional.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs =  torch.nn.functional.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


model = SimpleLanguageModel()
#m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iter):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iter - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, )
a = decoder(model.generate(context, max_new_tokens=2000)[0].tolist())
s = ''.join(a)
print(s)


0.433407 M parameters
step 0: train loss 5.6338, val loss 5.6357
step 100: train loss 2.6728, val loss 2.6740
step 200: train loss 1.9661, val loss 1.9788
step 300: train loss 1.0608, val loss 1.0887
step 400: train loss 0.5636, val loss 0.5941
step 500: train loss 0.3049, val loss 0.3278
step 600: train loss 0.2207, val loss 0.2334
step 700: train loss 0.1728, val loss 0.1850
step 800: train loss 0.1305, val loss 0.1420
step 900: train loss 0.1161, val loss 0.1211
step 999: train loss 0.1128, val loss 0.1169
 Údlllllllllllllllllyllyylllssed:uy, alveoelres,ueflvel, ryno, yey!o,
Pa ise rou,d,
S they yh.A:
Thetorredle, llre seou yo houre, veuyully hy hof,erl, proore healreo s,ulely derey yordere dorus yly e hillerp,hLNo our wyiecoudesifl, efroed seafy, urdureye dtheuredue,' chah,ull aket, mo cinouk y mheneorush;o, foh uriul yleos tulure herpewllevelerkereereGee:alss, eor ind,
Tisrt Etdehe mey regly,perale,Thlid,
MFrulereroe deuremer keeveuy oo ilceulld, actheeuv,
AGo noardvere, omhand-ou

In [71]:

# generate from the model
#$context = torch.zeros((1, 1), dtype=torch.long, )
#print(decoder(model.generate(context, max_new_tokens=1000)[0].tolist()).join())


 
Haaasllllbuuuuupueellllll;  isal llolliessot!
TYsho lio 'lgound o hirin gath datotat pcirizetshe t athe co atigace ha tathe onas
y he mus y hall;
Hast fifh thathre sol
 tou Iosn
ur tue, thie har do hst hath th hat bawh Dis h bimave fhad hayily it thray se es uithig b ato at taro prthatavathe hin wkigls awiury igt.
I ot oigut tha, te bou't; i hamit lanas
orict heis thiÃwill tÜy ut-thtti irathe tathe y Jtouur pe bldt haw wacthe ate to-tht hares un thary b toous
Bwhon'fer wamtigh to is ma moslse; tat isk si it ha thit bet
E(nh an kebr's ha boub, tibld!
Phes may?
C wetlphe is
Sishe atithit Sitt,
thive: tit meaisak sch st ath haseries th koofr to d nuwe liliss weaW say k wawi the masid
At thimet a sopeeangh theals apiest you ad, tha rsio thouÅuk isk ous im youdh ary ou arindrs h.

PEUK
TSho lethe y hi micapah reies at ot imeahatr ue oure atalle urs thist-
oinnd
ed the heant
2 wove y pir hith the y ast'e,
As
h haplee urs pod wis bl-hesn; ink toet ou l; thealausse h the:
Sbr; hat id 
he 